In [91]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [92]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import re

In [93]:
cid = 'ac248f45a5604f65baeed6d81ee23ede'
secret = '84c8de9f79784091b65f11a58c9240bd'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [94]:
def track_features(data):

    client_id = 'ac248f45a5604f65baeed6d81ee23ede'
    client_secret = '84c8de9f79784091b65f11a58c9240bd'

    auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    
    #audio features
    features = sp.audio_features(data)[0]
    
    #artist of the track, for genres and popularity
    artist = sp.track(data)["artists"][0]["id"]
    artist_pop = sp.artist(artist)["popularity"]
    artist_genres = sp.artist(artist)["genres"]
    
    #track popularity
    track_pop = sp.track(data)["popularity"]
    
    #add in extra features
    features["artist_pop"] = artist_pop
    if artist_genres:
        features["genres"] = " ".join([re.sub(' ','_',i) for i in artist_genres])
    else:
        features["genres"] = "unknown"
    features["track_pop"] = track_pop
    
    return features

In [95]:
def playlist_data(creator, playlist_id):
    
    #step1

    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    #step2
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # create empty dictionary
        playlist_features = {}
        
        # get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [96]:
track_df = playlist_data("vgrkmk46oax0lt10i9u6xnd67","2t2oW2uxY07bLid5B0XuVp")

In [97]:
track_df

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Stephen Sanchez,Until I Found You,Until I Found You,0T5iIrXA4p5GsubkhuBIKV,0.539,0.5080,10,-6.050,1,0.0288,0,0.1740,0.2270,101.358,177720,3
1,Peter McPoland,Romeo & Juliet,Romeo & Juliet,1VyA8mbSyXOdJx2lguR8Uu,0.318,0.4760,0,-7.108,1,0.0441,0.000016,0.5030,0.0807,160.737,166137,4
2,Montell Fish,Fall in Love with You.,Fall in Love with You.,31MQR9rOHRXyg4J01hRY1h,0.483,0.0394,4,-19.525,1,0.0537,0.0125,0.1120,0.1460,73.984,132000,4
3,Ali Gatie,YOU,It's You,5DqdesEfbRyOlSS3Tf6c29,0.732,0.4630,11,-6.972,0,0.0287,0,0.1940,0.3970,95.971,212607,4
4,Keane,Hopes And Fears,Somewhere Only We Know,0ll8uFnc0nANY35E0Lfxvg,0.453,0.6550,9,-4.220,1,0.0259,0.000008,0.0849,0.3680,172.012,236160,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Rex Orange County,WHO CARES?,THE SHADE,1gH1h30wkQdd9zhY3j7a8T,0.898,0.5090,5,-7.973,1,0.0620,0,0.0876,0.7270,119.869,181507,4
83,Rex Orange County,Pony,Pluto Projector,4EWBhKf1fOFnyMtUzACXEc,0.379,0.2190,5,-11.196,1,0.0307,0.000691,0.1180,0.1880,79.786,267520,4
84,Phoebe Bridgers,Punisher,Chinese Satellite,1scaOnahzG7o8pfwfsL784,0.301,0.4520,10,-9.931,1,0.0452,0.0219,0.1520,0.3630,120.040,217070,4
85,Novo Amor,Bathing Beach,Anchor,7qH9Z4dJEN0l9bidizW7fq,0.457,0.4070,9,-11.475,1,0.0308,0.884,0.1260,0.1260,117.053,257533,4


In [98]:
row = track_df.iloc[0]
print(row)

artist                     Stephen Sanchez
album                    Until I Found You
track_name               Until I Found You
track_id            0T5iIrXA4p5GsubkhuBIKV
danceability                         0.539
energy                               0.508
key                                     10
loudness                             -6.05
mode                                     1
speechiness                         0.0288
instrumentalness                         0
liveness                             0.174
valence                              0.227
tempo                              101.358
duration_ms                         177720
time_signature                           3
Name: 0, dtype: object


In [101]:
features_df = pd.DataFrame()

# get features for the track ids in playlist
for track_id in track_df['track_id']:
    features = track_features(track_id)
    
    features_df_to_concat = pd.DataFrame([features])
    
    features_df = pd.concat([features_df_to_concat, features_df], ignore_index=True)

In [102]:
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_pop,genres,track_pop
0,0.518,0.4900,8,-7.437,1,0.0281,0.4040,0.037700,0.1120,0.2480,79.992,audio_features,5UhZyWfrSEhGYfIIFNYscF,spotify:track:5UhZyWfrSEhGYfIIFNYscF,https://api.spotify.com/v1/tracks/5UhZyWfrSEhG...,https://api.spotify.com/v1/audio-analysis/5UhZ...,210203,4,56,alt_z bedroom_pop pov:_indie,56
1,0.457,0.4070,9,-11.475,1,0.0308,0.8050,0.884000,0.1260,0.1260,117.053,audio_features,7qH9Z4dJEN0l9bidizW7fq,spotify:track:7qH9Z4dJEN0l9bidizW7fq,https://api.spotify.com/v1/tracks/7qH9Z4dJEN0l...,https://api.spotify.com/v1/audio-analysis/7qH9...,257533,4,70,ambient_folk indie_anthem-folk indie_folk mode...,79
2,0.301,0.4520,10,-9.931,1,0.0452,0.7420,0.021900,0.1520,0.3630,120.040,audio_features,1scaOnahzG7o8pfwfsL784,spotify:track:1scaOnahzG7o8pfwfsL784,https://api.spotify.com/v1/tracks/1scaOnahzG7o...,https://api.spotify.com/v1/audio-analysis/1sca...,217070,4,78,indie_pop la_indie pov:_indie,63
3,0.379,0.2190,5,-11.196,1,0.0307,0.7070,0.000691,0.1180,0.1880,79.786,audio_features,4EWBhKf1fOFnyMtUzACXEc,spotify:track:4EWBhKf1fOFnyMtUzACXEc,https://api.spotify.com/v1/tracks/4EWBhKf1fOFn...,https://api.spotify.com/v1/audio-analysis/4EWB...,267520,4,74,bedroom_pop,79
4,0.898,0.5090,5,-7.973,1,0.0620,0.3910,0.000000,0.0876,0.7270,119.869,audio_features,1gH1h30wkQdd9zhY3j7a8T,spotify:track:1gH1h30wkQdd9zhY3j7a8T,https://api.spotify.com/v1/tracks/1gH1h30wkQdd...,https://api.spotify.com/v1/audio-analysis/1gH1...,181507,4,74,bedroom_pop,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,0.453,0.6550,9,-4.220,1,0.0259,0.0794,0.000008,0.0849,0.3680,172.012,audio_features,0ll8uFnc0nANY35E0Lfxvg,spotify:track:0ll8uFnc0nANY35E0Lfxvg,https://api.spotify.com/v1/tracks/0ll8uFnc0nAN...,https://api.spotify.com/v1/audio-analysis/0ll8...,236160,4,72,neo_mellow piano_rock pop_rock,77
83,0.732,0.4630,11,-6.972,0,0.0287,0.3740,0.000000,0.1940,0.3970,95.971,audio_features,5DqdesEfbRyOlSS3Tf6c29,spotify:track:5DqdesEfbRyOlSS3Tf6c29,https://api.spotify.com/v1/tracks/5DqdesEfbRyO...,https://api.spotify.com/v1/audio-analysis/5Dqd...,212607,4,67,canadian_hip_hop,80
84,0.483,0.0394,4,-19.525,1,0.0537,0.9220,0.012500,0.1120,0.1460,73.984,audio_features,31MQR9rOHRXyg4J01hRY1h,spotify:track:31MQR9rOHRXyg4J01hRY1h,https://api.spotify.com/v1/tracks/31MQR9rOHRXy...,https://api.spotify.com/v1/audio-analysis/31MQ...,132000,4,73,pittsburgh_rap,0
85,0.318,0.4760,0,-7.108,1,0.0441,0.4540,0.000016,0.5030,0.0807,160.737,audio_features,1VyA8mbSyXOdJx2lguR8Uu,spotify:track:1VyA8mbSyXOdJx2lguR8Uu,https://api.spotify.com/v1/tracks/1VyA8mbSyXOd...,https://api.spotify.com/v1/audio-analysis/1VyA...,166137,4,56,pov:_indie,63


In [103]:
track_df

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Stephen Sanchez,Until I Found You,Until I Found You,0T5iIrXA4p5GsubkhuBIKV,0.539,0.5080,10,-6.050,1,0.0288,0,0.1740,0.2270,101.358,177720,3
1,Peter McPoland,Romeo & Juliet,Romeo & Juliet,1VyA8mbSyXOdJx2lguR8Uu,0.318,0.4760,0,-7.108,1,0.0441,0.000016,0.5030,0.0807,160.737,166137,4
2,Montell Fish,Fall in Love with You.,Fall in Love with You.,31MQR9rOHRXyg4J01hRY1h,0.483,0.0394,4,-19.525,1,0.0537,0.0125,0.1120,0.1460,73.984,132000,4
3,Ali Gatie,YOU,It's You,5DqdesEfbRyOlSS3Tf6c29,0.732,0.4630,11,-6.972,0,0.0287,0,0.1940,0.3970,95.971,212607,4
4,Keane,Hopes And Fears,Somewhere Only We Know,0ll8uFnc0nANY35E0Lfxvg,0.453,0.6550,9,-4.220,1,0.0259,0.000008,0.0849,0.3680,172.012,236160,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Rex Orange County,WHO CARES?,THE SHADE,1gH1h30wkQdd9zhY3j7a8T,0.898,0.5090,5,-7.973,1,0.0620,0,0.0876,0.7270,119.869,181507,4
83,Rex Orange County,Pony,Pluto Projector,4EWBhKf1fOFnyMtUzACXEc,0.379,0.2190,5,-11.196,1,0.0307,0.000691,0.1180,0.1880,79.786,267520,4
84,Phoebe Bridgers,Punisher,Chinese Satellite,1scaOnahzG7o8pfwfsL784,0.301,0.4520,10,-9.931,1,0.0452,0.0219,0.1520,0.3630,120.040,217070,4
85,Novo Amor,Bathing Beach,Anchor,7qH9Z4dJEN0l9bidizW7fq,0.457,0.4070,9,-11.475,1,0.0308,0.884,0.1260,0.1260,117.053,257533,4


In [104]:
columns_to_drop = ["danceability", "energy", "key", "loudness", "mode", "speechiness", "instrumentalness", 
                   "liveness", "valence", "tempo", "duration_ms", "time_signature"]
track_df = track_df.drop(columns=columns_to_drop)

In [105]:
track_df

,artist,album,track_name,track_id
0,Stephen Sanchez,Until I Found You,Until I Found You,0T5iIrXA4p5GsubkhuBIKV
1,Peter McPoland,Romeo & Juliet,Romeo & Juliet,1VyA8mbSyXOdJx2lguR8Uu
2,Montell Fish,Fall in Love with You.,Fall in Love with You.,31MQR9rOHRXyg4J01hRY1h
3,Ali Gatie,YOU,It's You,5DqdesEfbRyOlSS3Tf6c29
4,Keane,Hopes And Fears,Somewhere Only We Know,0ll8uFnc0nANY35E0Lfxvg
...,...,...,...,...
82,Rex Orange County,WHO CARES?,THE SHADE,1gH1h30wkQdd9zhY3j7a8T
83,Rex Orange County,Pony,Pluto Projector,4EWBhKf1fOFnyMtUzACXEc
84,Phoebe Bridgers,Punisher,Chinese Satellite,1scaOnahzG7o8pfwfsL784
85,Novo Amor,Bathing Beach,Anchor,7qH9Z4dJEN0l9bidizW7fq


In [106]:
df = pd.merge(track_df, features_df, left_on = "track_id", right_on = "id")

In [108]:
pd.set_option('display.max_columns', None)
columns_to_drop1 = ["type", "id", "uri"]
df = df.drop(columns=columns_to_drop1)
df

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_href,analysis_url,duration_ms,time_signature,artist_pop,genres,track_pop
0,Stephen Sanchez,Until I Found You,Until I Found You,0T5iIrXA4p5GsubkhuBIKV,0.539,0.5080,10,-6.050,1,0.0288,0.7270,0.000000,0.1740,0.2270,101.358,https://api.spotify.com/v1/tracks/0T5iIrXA4p5G...,https://api.spotify.com/v1/audio-analysis/0T5i...,177720,3,75,gen_z_singer-songwriter,83
1,Peter McPoland,Romeo & Juliet,Romeo & Juliet,1VyA8mbSyXOdJx2lguR8Uu,0.318,0.4760,0,-7.108,1,0.0441,0.4540,0.000016,0.5030,0.0807,160.737,https://api.spotify.com/v1/tracks/1VyA8mbSyXOd...,https://api.spotify.com/v1/audio-analysis/1VyA...,166137,4,56,pov:_indie,63
2,Montell Fish,Fall in Love with You.,Fall in Love with You.,31MQR9rOHRXyg4J01hRY1h,0.483,0.0394,4,-19.525,1,0.0537,0.9220,0.012500,0.1120,0.1460,73.984,https://api.spotify.com/v1/tracks/31MQR9rOHRXy...,https://api.spotify.com/v1/audio-analysis/31MQ...,132000,4,73,pittsburgh_rap,0
3,Ali Gatie,YOU,It's You,5DqdesEfbRyOlSS3Tf6c29,0.732,0.4630,11,-6.972,0,0.0287,0.3740,0.000000,0.1940,0.3970,95.971,https://api.spotify.com/v1/tracks/5DqdesEfbRyO...,https://api.spotify.com/v1/audio-analysis/5Dqd...,212607,4,67,canadian_hip_hop,80
4,Keane,Hopes And Fears,Somewhere Only We Know,0ll8uFnc0nANY35E0Lfxvg,0.453,0.6550,9,-4.220,1,0.0259,0.0794,0.000008,0.0849,0.3680,172.012,https://api.spotify.com/v1/tracks/0ll8uFnc0nAN...,https://api.spotify.com/v1/audio-analysis/0ll8...,236160,4,72,neo_mellow piano_rock pop_rock,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Rex Orange County,WHO CARES?,THE SHADE,1gH1h30wkQdd9zhY3j7a8T,0.898,0.5090,5,-7.973,1,0.0620,0.3910,0.000000,0.0876,0.7270,119.869,https://api.spotify.com/v1/tracks/1gH1h30wkQdd...,https://api.spotify.com/v1/audio-analysis/1gH1...,181507,4,74,bedroom_pop,80
83,Rex Orange County,Pony,Pluto Projector,4EWBhKf1fOFnyMtUzACXEc,0.379,0.2190,5,-11.196,1,0.0307,0.7070,0.000691,0.1180,0.1880,79.786,https://api.spotify.com/v1/tracks/4EWBhKf1fOFn...,https://api.spotify.com/v1/audio-analysis/4EWB...,267520,4,74,bedroom_pop,79
84,Phoebe Bridgers,Punisher,Chinese Satellite,1scaOnahzG7o8pfwfsL784,0.301,0.4520,10,-9.931,1,0.0452,0.7420,0.021900,0.1520,0.3630,120.040,https://api.spotify.com/v1/tracks/1scaOnahzG7o...,https://api.spotify.com/v1/audio-analysis/1sca...,217070,4,78,indie_pop la_indie pov:_indie,63
85,Novo Amor,Bathing Beach,Anchor,7qH9Z4dJEN0l9bidizW7fq,0.457,0.4070,9,-11.475,1,0.0308,0.8050,0.884000,0.1260,0.1260,117.053,https://api.spotify.com/v1/tracks/7qH9Z4dJEN0l...,https://api.spotify.com/v1/audio-analysis/7qH9...,257533,4,70,ambient_folk indie_anthem-folk indie_folk mode...,79


In [110]:
df.to_csv('../spotifyrecommendationsystem/processed_data.csv')